In [ ]:
using PyPlot, ProgressMeter
using Random, LinearAlgebra

Experiments with the Dirichlet kernel. The Dirichlet filter is $\psi(x)=\sum_{k=-nf}^{nf} \exp( \sqrt{-1}kx)$. The Hilbert space is $L^2$ on the torus and the loss 
$$
R(f) = \frac12 \frac1{2\pi}\int_0^{2\pi}\vert f(x)- f^\star(x)\vert^2 d x
$$
Here we consider $f(x)=\int \psi(x-\theta)d \mu(\theta)$ which leads to a translation invariant kernel
$$
R\left(\int \psi d \mu\right) = \frac12\iint k(\theta,\theta') d\mu(\theta)d\mu(\theta') +  \frac12\iint k(\theta,\theta') d\mu^\star(\theta)d\mu^\star(\theta') - \iint  k(\theta,\theta') d\mu(\theta)d\mu^\star(\theta')
$$
where $k(\theta,\theta')=\frac1{2\pi}\int \psi(x-\theta)\bar \psi(x-\theta')dx = \psi(\theta'-\theta)$ by direct computations.

In [ ]:
nf = 2 # number of frequency components
phi(x) = real(sum(exp.(im*k*x) for k in -nf:nf))
#phi_der(x) = real(sum(im*k*exp.(im*k*x) for k in -nf:nf))

In [ ]:
function sfth(x,t) #soft-thresholding
    sign(x) * max(0.0, abs(x) - t)
end

In [ ]:
"""
PGM for 1D sparse deconvolution. θ_init is the grid, (a_obs, θ_obs) is the ground truth.
"""
# is p>1 then usual power entropy, if p=1 this is entropy if signed=false or hypentropy if signed=true
function PGM_deconv_1D(θ_init, a_obs, θ_obs, lambda, stepsize, niter; p=1, signed=false)
    m0   = length(a_obs)
    m    = length(θ_init)
    as   = zeros(m, niter)
    a    = (signed ? zeros(m) : ones(m))
    loss = zeros(niter)
    Kxx  = phi.(θ_init .- θ_init')
    Kyy  = phi.(θ_obs .- θ_obs')
    Kxy  = phi.(θ_init .- θ_obs')

    @showprogress 1 "Computing..." for iter = 1:niter
        as[:,iter] = a
        loss[iter] = (a' * Kxx * a /m^2 + a_obs' * Kyy * a_obs /(m0*m0) - 2a' * Kxy * a_obs/(m*m0) )/2 + lambda * sum(abs.(a))/m
        gradG = Kxx * a/m .- Kxy * a_obs/m0
        if signed
            if p>1
                temp = sfth.( sign.(a).*abs.(a).^(p-1) /(p-1) - stepsize * gradG, stepsize*lambda)
                a    = (p-1)^(1/(p-1)) * sign.(temp) .* abs.(temp).^(1/(p-1))
            else
                a = sinh.(sfth.( asinh.(a) - stepsize * gradG, stepsize*lambda))
            end
        else
            if p>1
                a = (p-1)^(1/(p-1)) * max.(0.0, a.^(p-1) /(p-1) - stepsize * (gradG .+ lambda)).^(1/(p-1))
            else
                a = a .* exp.( - stepsize * (gradG .+ lambda) )
            end
        end
    end
    return as, loss
end

In [ ]:
"""
APGM for 1D sparse deconvolution. θ_init is the grid, (a_obs, θ_obs) is the ground truth.
"""
# is p>1 then usual power entropy, if p=1 this is entropy if signed=false or hypentropy if signed=true
function APGM_deconv_1D(θ_init, a_obs, θ_obs, lambda, stepsize, niter; p=1, signed=false)
    m0   = length(a_obs)
    m    = length(θ_init)
    as   = zeros(m, niter)
    a    = (signed ? zeros(m) : ones(m))
    b    = copy(a)
    c    = copy(a)
    loss = zeros(niter)
    Kxx  = phi.(θ_init .- θ_init')
    Kyy  = phi.(θ_obs .- θ_obs')
    Kxy  = phi.(θ_init .- θ_obs')
    theta=1.0

    @showprogress 1 "Computing..." for iter = 1:niter
        as[:,iter] = a
        loss[iter] = (a' * Kxx * a /m^2 + a_obs' * Kyy * a_obs /(m0*m0) - 2a' * Kxy * a_obs/(m*m0) )/2 + lambda * sum(abs.(a))/m
        b = (1-theta)*a .+ theta*c
        gradG = Kxx * b/m .- Kxy * a_obs/m0
        if signed
            if p>1
                temp = sfth.( sign.(c).*abs.(c).^(p-1) /(p-1) - (stepsize/theta) * gradG, (stepsize/theta)*lambda)
                c    = (p-1)^(1/(p-1)) * sign.(temp) .* abs.(temp).^(1/(p-1))
            else
                c = sinh.(sfth.( asinh.(c) - (stepsize/theta) * gradG, (stepsize/theta)*lambda))
            end
        else
            if p>1
                c = (p-1)^(1/(p-1)) * max.(0.0, c.^(p-1) /(p-1) - (stepsize/theta) * (gradG .+ lambda)).^(1/(p-1))
            else
                c = c .* exp.( - (stepsize/theta) * (gradG .+ lambda) )
            end
        end
        a = (1-theta)*a .+ theta*c
        theta = (sqrt(theta^2+4)-theta)*theta/2
    end
    return as, loss
end

# Nonnegative case

In [ ]:
m0=2
#min_separation = 0.2
#@assert m0 * min_separation < π/2 # they can't be all far away
#θ0 = sort(rand(m0)) * 2π # generate random spikes on the domain [0,2π]
#while min(minimum(abs.(θ0[2:end] - θ0[1:end-1])), abs((θ0[end]-2π)-θ0[1])) < min_separation # min separation
#    θ0 = sort(rand(m0)) * 2π
#end
#a_obs = (1 .+ 0.0*rand(m0))/2#normalize(sign.(randn(m0))/2 + (rand(m0) .- 1/2)/2 , 1)

# learn 1 spike
m0 = 1
θ0 = [π]
a_obs = [1]


stepsize = 0.1
lambda = 0.0#5
θ_obs = θ0
m = 300
θ_init = range(0,2π*(1-1/m),length=m)
niter = 10000
asA, lossA  = PGM_deconv_1D(θ_init, a_obs, θ_obs, lambda, stepsize, niter; p=1, signed=false);
asB, lossB  = PGM_deconv_1D(θ_init, a_obs, θ_obs, lambda, stepsize, niter; p=1.5, signed=false);
asC, lossC  = PGM_deconv_1D(θ_init, a_obs, θ_obs, lambda, stepsize, niter; p=2, signed=false);
asD, lossD  = PGM_deconv_1D(θ_init, a_obs, θ_obs, lambda, stepsize, niter; p=4, signed=false);
asAA, lossAA  = APGM_deconv_1D(θ_init, a_obs, θ_obs, lambda, stepsize, niter; p=1, signed=false);
asBB, lossBB  = APGM_deconv_1D(θ_init, a_obs, θ_obs, lambda, stepsize, niter; p=1.5, signed=false);
asCC, lossCC  = APGM_deconv_1D(θ_init, a_obs, θ_obs, lambda, stepsize, niter; p=2, signed=false);

In [ ]:
figure(figsize=[14,2.5])
cmap = ColorMap("viridis")
ts = (6).^(0:4)
#θ_init =θ_init ./2π
subplot(141)
for t=1:length(ts)
    fill_between(θ_init,asA[:,ts[t]],color=cmap(1.2-t/length(ts)),alpha=0.4)
end
axis([0,1,0,16])
axis("off")
ylabel(L"Density of $f_k$")
title(L"p=1")

subplot(142)
for t=1:length(ts)
    fill_between(θ_init,asB[:,ts[t]],color=cmap(1.2-t/length(ts)),alpha=0.4)
end
axis([0,1,0,15])
title(L"p=1.5")
axis("off")
subplot(143)
for t=1:length(ts)
    fill_between(θ_init,asC[:,ts[t]],color=cmap(1.2-t/length(ts)),alpha=0.4)
end
axis([0,1,0,15])
title(L"p=2")
axis("off")
subplot(144)
for t=1:length(ts)
    fill_between(θ_init,asD[:,ts[t]],color=cmap(1.2-t/length(ts)),alpha=0.4)
end
axis([0,1,0,15])
title(L"p=4")
axis("off")
savefig("deconv_1D_plot_PGM.png",bbox_inches="tight",dpi=200)

In [ ]:
figure(figsize=[4,3])
loglog(lossAA,"C0",label=L"p=1") 
trgx = 1000*[1; 10; 10; 1]
trgy = copy(trgx)
trgy[3] = trgx[1]
loglog(lossBB,"C1",label=L"p=1.5")
loglog(lossCC,"C3",label=L"p=2") 
loglog(trgx, trgy.^(-2)*(lossAA[1000]/trgy[1].^(-2)) ,"--C0",label=L"k^{-2}")
loglog(trgx, trgy.^(-8/4.5)*(lossBB[1000]/trgy[1].^(-8/4.5)),"--C1",label=L"k^{-8/4.5}")
loglog(trgx, trgy.^(-8/5)*(lossCC[1000]/trgy[1].^(-8/5)) ,"--C3",label=L"k^{-8/5}")

legend(loc=3,ncol=2)
xlabel(L"k")
ylabel(L"F(f_k)-\inf\; F")
grid("on")
savefig("deconv_1D_pos_APGM.png",bbox_inches="tight",dpi=200)

In [ ]:
figure(figsize=[4,3])
loglog(lossA,"C0",label=L"p=1") 
#plot([100; 200; 200; 100], 1.0./[100 ;200; 100; 100],"--C0")
trgx = 100*[10; 100; 100; 10]
trgy = copy(trgx)
trgy[3] = trgx[1]
loglog(lossB,"C1",label=L"p=1.5")
loglog(lossC,"C3",label=L"p=2") 
loglog(trgx, trgy.^(-1)*(lossA[1000]/trgy[1].^(-1)) ,"--C0",label=L"k^{-1}")
loglog(trgx, trgy.^(-4/4.5)*(lossB[1000]/trgy[1].^(-4/4.5))  ,"--C1",label=L"k^{-4/4.5}")
loglog(trgx, trgy.^(-4/5)*(lossC[1000]/trgy[1].^(-4/5))  ,"--C3",label=L"k^{-4/5}")

legend(ncol=2)
xlabel(L"k")
ylabel(L"F(f_k)-\inf\; F")
grid("on")
savefig("deconv_1D_pos_PGM.png",bbox_inches="tight",dpi=200)

# Signed case

In [ ]:
# learn 1 spike
m0 = 1
θ0 = [π]
a_obs = [1]


stepsize = 0.1
lambda = 0.5
θ_obs = θ0
m = 1000
#θ_init = θ0
θ_init = range(0,2π*(1-1/m),length=m)
niter = 10000
asA, lossA  = PGM_deconv_1D(θ_init, a_obs, θ_obs, lambda, stepsize, niter; p=1, signed=true);
asB, lossB  = PGM_deconv_1D(θ_init, a_obs, θ_obs, lambda, stepsize, niter; p=1.5, signed=true);
asC, lossC  = PGM_deconv_1D(θ_init, a_obs, θ_obs, lambda, stepsize, niter; p=2, signed=true);
asAA, lossAA  = APGM_deconv_1D(θ_init, a_obs, θ_obs, lambda, stepsize, niter; p=1, signed=true);
asBB, lossBB  = APGM_deconv_1D(θ_init, a_obs, θ_obs, lambda, stepsize, niter; p=1.5, signed=true);
asCC, lossCC  = APGM_deconv_1D(θ_init, a_obs, θ_obs, lambda, stepsize, niter; p=2, signed=true);

In [ ]:
figure(figsize=[4,3])
opt = lambda*(1-lambda/(4nf+2))
loglog(lossA .- opt ,"C0",label=L"p=1") 
trgx = 100*[10; 100; 100; 10]
trgy = copy(trgx)
trgy[3] = trgx[1]
loglog(lossB.- opt ,"C1",label=L"p=1.5")
loglog(lossC.- opt ,"C3",label=L"p=2") 
loglog(trgx, trgy.^(-1)*((lossA[1000]- opt)/trgy[1]^(-1)) ,"--C0",label=L"k^{-1}")
loglog(trgx, trgy.^(-2/2.5)*((lossB[1000]- opt)/trgy[1]^(-2/2.5)) ,"--C1",label=L"k^{-2/2.5}")
loglog(trgx, trgy.^(-2/3)*((lossC[1000]- opt)/trgy[1]^(-2/3)) ,"--C3",label=L"k^{-2/3}")

legend(loc=3,ncol=2)
xlabel(L"k")
ylabel(L"F(f_k)-\inf\; F")
grid("on")
savefig("deconv_1D_sign_PGM.png",bbox_inches="tight",dpi=200)

In [ ]:
figure(figsize=[4,3])
loglog(lossAA .- opt,"C0",label=L"p=1") 
trgx = 1000*[1; 10; 10; 1]
trgy = copy(trgx)
trgy[3] = trgx[1]
loglog(lossBB .- opt,"C1",label=L"p=1.5")
loglog(lossCC .- opt,"C3",label=L"p=2") 
loglog(trgx, trgy.^(-2)*((lossAA[1000]- opt)/trgy[1]^(-2)),"--C0",label=L"k^{-2}")
loglog(trgx, trgy.^(-4/2.5)*((lossBB[1000]- opt)/trgy[1]^(-4/2.5)) ,"--C1",label=L"k^{-4/2.5}")
loglog(trgx, trgy.^(-4/3)*((lossCC[1000]- opt)/trgy[1]^(-4/3)) ,"--C3",label=L"k^{-4/3}")

legend(loc=3,ncol=2)
xlabel(L"k")
ylabel(L"F(f_k)-\inf\; F")
grid("on")
savefig("deconv_1D_sign_APGM.png",bbox_inches="tight",dpi=200)

# 2-d case

In [ ]:
nf = 2 # number of frequency components
phi(x,y) = real(sum(exp(im*(kx*x + ky*y)) for kx in -nf:nf, ky in -nf:nf))

In [ ]:
"""
PGM for 2D sparse deconvolution. θ_init is the grid, (a_obs, θ_obs) is the ground truth.
"""
# is p>1 then usual power entropy, if p=1 this is entropy if signed=false or hypentropy if signed=true
function PGM_deconv_2D(θ_init, a_obs, θ_obs, lambda, stepsize, niter; p=1, signed=false)
    m0   = length(a_obs)
    m    = size(θ_init,1)
    as   = zeros(m, niter)
    a    = (signed ? zeros(m) : ones(m))
    loss = zeros(niter)
    Kxx = phi.(θ_init[:,1] .- θ_init[:,1]',θ_init[:,2] .- θ_init[:,2]')
    Kyy = phi.(θ_obs[:,1] .- θ_obs[:,1]',θ_obs[:,2] .- θ_obs[:,2]')
    Kxy = phi.(θ_init[:,1] .- θ_obs[:,1]', θ_init[:,2] .- θ_obs[:,2]')

    @showprogress 1 "Computing..." for iter = 1:niter
        as[:,iter] = a
        loss[iter] = (a' * Kxx * a /m^2 + a_obs' * Kyy * a_obs /(m0*m0) - 2a' * Kxy * a_obs/(m*m0) )/2 + lambda * sum(abs.(a))/m
        gradG = Kxx * a/m .- Kxy * a_obs/m0
        if signed
            if p>1
                temp = sfth.( sign.(a).*abs.(a).^(p-1) /(p-1) - stepsize * gradG, stepsize*lambda)
                a    = (p-1)^(1/(p-1)) * sign.(temp) .* abs.(temp).^(1/(p-1))
            else
                a = sinh.(sfth.( asinh.(a) - stepsize * gradG, stepsize*lambda))
            end
        else
            if p>1
                a = (p-1)^(1/(p-1)) * max.(0.0, a.^(p-1) /(p-1) - stepsize * (gradG .+ lambda)).^(1/(p-1))
            else
                a = a .* exp.( - stepsize * (gradG .+ lambda) )
            end
        end
    end
    return as, loss
end

In [ ]:
"""
APGM for 2D sparse deconvolution. θ_init is the grid, (a_obs, θ_obs) is the ground truth.
"""
# is p>1 then usual power entropy, if p=1 this is entropy if signed=false or hypentropy if signed=true
function APGM_deconv_2D(θ_init, a_obs, θ_obs, lambda, stepsize, niter; p=1, signed=false)
    m0   = length(a_obs)
    m    = size(θ_init,1)
    as   = zeros(m, niter)
    a    = (signed ? zeros(m) : ones(m))
    b    = copy(a)
    c    = copy(a)
    loss = zeros(niter)
    Kxx = phi.(θ_init[:,1] .- θ_init[:,1]',θ_init[:,2] .- θ_init[:,2]')
    Kyy = phi.(θ_obs[:,1] .- θ_obs[:,1]',θ_obs[:,2] .- θ_obs[:,2]')
    Kxy = phi.(θ_init[:,1] .- θ_obs[:,1]', θ_init[:,2] .- θ_obs[:,2]')
    theta=1.0

    @showprogress 1 "Computing..." for iter = 1:niter
        as[:,iter] = a
        loss[iter] = (a' * Kxx * a /m^2 + a_obs' * Kyy * a_obs /(m0*m0) - 2a' * Kxy * a_obs/(m*m0) )/2 + lambda * sum(abs.(a))/m
        b = (1-theta)*a .+ theta*c
        gradG = Kxx * b/m .- Kxy * a_obs/m0
        if signed
            if p>1
                temp = sfth.( sign.(c).*abs.(c).^(p-1) /(p-1) - (stepsize/theta) * gradG, (stepsize/theta)*lambda)
                c    = (p-1)^(1/(p-1)) * sign.(temp) .* abs.(temp).^(1/(p-1))
            else
                c = sinh.(sfth.( asinh.(c) - (stepsize/theta) * gradG, (stepsize/theta)*lambda))
            end
        else
            if p>1
                c = (p-1)^(1/(p-1)) * max.(0.0, c.^(p-1) /(p-1) - (stepsize/theta) * (gradG .+ lambda)).^(1/(p-1))
            else
                c = c .* exp.( - (stepsize/theta) * (gradG .+ lambda) )
            end
        end
        a = (1-theta)*a .+ theta*c
        theta = (sqrt(theta^2+4)-theta)*theta/2
    end
    return as, loss
end

In [ ]:
Random.seed!(1) # randomness seed # best was 2,4
m0 = 1 # number of spikes ground truth
θ0 = [π, π]'
a_obs  = [1]
θ_obs = θ0

res = 60 # resolution of the initial measure
m = res^2

θ_initx = range(π/res, 2π - π/res, length=res)' .* ones(res)
θ_inity = range(π/res, 2π - π/res, length=res) .* ones(res)'
θ_init  = cat(θ_initx[:], θ_inity[:], dims=2)
niter = 10000
stepsize = 0.01
lambda =  0.0
asA, lossA = PGM_deconv_2D(θ_init, a_obs, θ_obs, lambda, stepsize, niter; p=1, signed=false);
asB, lossB = PGM_deconv_2D(θ_init, a_obs, θ_obs, lambda, stepsize, niter; p=1.5, signed=false);
asC, lossC = PGM_deconv_2D(θ_init, a_obs, θ_obs, lambda, stepsize, niter; p=2, signed=false);
asAA, lossAA = APGM_deconv_2D(θ_init, a_obs, θ_obs, lambda, stepsize, niter; p=1, signed=false);
asBB, lossBB = APGM_deconv_2D(θ_init, a_obs, θ_obs, lambda, stepsize, niter; p=1.5, signed=false);
asCC, lossCC = APGM_deconv_2D(θ_init, a_obs, θ_obs, lambda, stepsize, niter; p=2, signed=false);

In [ ]:
figure(figsize=[4,3])
loglog(lossA,"C0",label=L"p=1") 
trgx = 100*[10; 100; 100; 10]
trgy = copy(trgx)
trgy[3] = trgx[1]
loglog(lossB,"C1",label=L"p=1.5")
loglog(lossC,"C3",label=L"p=2") 
loglog(trgx, trgy.^(-1)*(lossA[1000]/trgy[1]^(-1)) ,"--C0",label=L"k^{-1}")
loglog(trgx, trgy.^(-4/5)*(lossB[1000]/trgy[1]^(-4/5)) ,"--C1",label=L"k^{-4/5}")
loglog(trgx, trgy.^(-4/6)*(lossC[1000]/trgy[1]^(-4/6)) ,"--C3",label=L"k^{-4/6}")

legend(loc=3,ncol=2)
xlabel(L"k")
ylabel(L"F(f_k)-\inf\; F")
grid("on")
savefig("deconv_2D_pos_PGM.png",bbox_inches="tight",dpi=200)

In [ ]:
figure(figsize=[4,3])
loglog(lossAA[1:10000],"C0",label=L"p=1") 
trgx = 10*[10; 100; 100; 10]
trgy = copy(trgx)
trgy[3] = trgx[1]
loglog(lossBB[1:10000],"C1",label=L"p=1.5")
loglog(lossCC[1:10000],"C3",label=L"p=2") 
loglog(trgx, trgy.^(-2)*(lossAA[100]/trgy[1]^(-2)) ,"--C0",label=L"k^{-2}")
loglog(trgx, trgy.^(-8/5)*(lossBB[100]/trgy[1]^(-8/5)) ,"--C1",label=L"k^{-8/5}")
loglog(trgx, trgy.^(-8/6)*(lossCC[100]/trgy[1]^(-8/6)) ,"--C3",label=L"k^{-8/6}")

legend(loc=3,ncol=2)
xlabel(L"k")
ylabel(L"F(f_k)-\inf\; F")
grid("on")
savefig("deconv_2D_pos_APGM.png",bbox_inches="tight",dpi=200)

# Signed 2d

In [ ]:
Random.seed!(1) # randomness seed # best was 2,4
m0 = 1 # number of spikes ground truth
θ0 = [π, π]'
a_obs  = [1]
θ_obs = θ0

res = 60 # resolution of the initial measure
m = res^2

θ_initx = range(π/res, 2π - π/res, length=res)' .* ones(res)
θ_inity = range(π/res, 2π - π/res, length=res) .* ones(res)'
θ_init  = cat(θ_initx[:], θ_inity[:], dims=2)
niter = 10000
stepsize = 0.01
lambda =  0.5
asA, lossA = PGM_deconv_2D(θ_init, a_obs, θ_obs, lambda, stepsize, niter; p=1, signed=true);
asB, lossB = PGM_deconv_2D(θ_init, a_obs, θ_obs, lambda, stepsize, niter; p=1.5, signed=true);
asC, lossC = PGM_deconv_2D(θ_init, a_obs, θ_obs, lambda, stepsize, niter; p=2, signed=true);
asAA, lossAA = APGM_deconv_2D(θ_init, a_obs, θ_obs, lambda, stepsize, niter; p=1, signed=true);
asBB, lossBB = APGM_deconv_2D(θ_init, a_obs, θ_obs, lambda, stepsize, niter; p=1.5, signed=true);
asCC, lossCC = APGM_deconv_2D(θ_init, a_obs, θ_obs, lambda, stepsize, niter; p=2, signed=true);

In [ ]:
opt = lambda*(1-lambda/2(2nf+1)^2); # closed form optimal value

In [ ]:
figure(figsize=[4,3])
loglog(lossA .- opt,"C0",label=L"p=1") 
trgx = 100*[10; 100; 100; 10]
trgy = copy(trgx)
trgy[3] = trgx[1]
loglog(lossB .- opt,"C1",label=L"p=1.5")
loglog(lossC .- opt,"C3",label=L"p=2") 
loglog(trgx, trgy.^(-1)*((lossA[1000]-opt)/trgy[1].^(-1)) ,"--C0",label=L"k^{-1}")
loglog(trgx, trgy.^(-2/3)*((lossB[10000]-opt)/trgy[2].^(-2/3)) ,"--C1",label=L"k^{-2/3}")
loglog(trgx, trgy.^(-2/4)*((lossC[10000]-opt)/trgy[2].^(-2/4)) ,"--C3",label=L"k^{-2/4}")

legend(loc=3,ncol=2)
xlabel(L"k")
ylabel(L"F(f_k)-\inf\; F")
grid("on")
savefig("deconv_2D_sign_PGM.png",bbox_inches="tight",dpi=200)

In [ ]:
figure(figsize=[4,3])
loglog(lossAA[1:10000] .- opt,"C0",label=L"p=1") 
trgx = 10*[10; 100; 100; 10]
trgy = copy(trgx)
trgy[3] = trgx[1]
loglog(lossBB[1:10000] .- opt,"C1",label=L"p=1.5")
loglog(lossCC[1:10000] .- opt,"C3",label=L"p=2") 
loglog(trgx, trgy.^(-2)*((lossAA[100]-opt)/trgy[1].^(-2)) ,"--C0",label=L"k^{-2}")
loglog(trgx, trgy.^(-4/3)*((lossBB[1000]-opt)/trgy[2].^(-4/3)) ,"--C1",label=L"k^{-4/3}")
loglog(trgx, trgy.^(-4/4)*((lossCC[1000]-opt)/trgy[2].^(-4/4)) ,"--C3",label=L"k^{-4/4}")

legend(loc=3,ncol=2)
xlabel(L"k")
ylabel(L"F(f_k)-\inf\; F")
grid("on")
savefig("deconv_2D_sign_APGM.png",bbox_inches="tight",dpi=200)